# 经典的Strategy Pattern

In [19]:
from abc import ABCMeta, abstractmethod
from collections import namedtuple

In [20]:
Customer = namedtuple('Customer', 'name fidelity')

In [21]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

In [22]:
class Order: # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self) # Promotion类方法discount()需要接收一个Order对象作为参数
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [23]:
class Promotion(ABCMeta): # the Strategy: an Abstract Base Class
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""
        
    

In [24]:
class FidelityPromo(Promotion): # first Concrete Strategy
    """5% discount for customers with 1000 or more fideliy points"""
    def discount(self, order):
        return order.total() * .5 if order.customer.fidelity >= 1000 else 0
    
    

In [25]:
class BulkItemPromo(Promotion): # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount
    

In [26]:
class LargeOrderPromo(Promotion): # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    def discount(self, order):
        distinct_items = {item.product for item in item.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

# Function-oriented Strategy Pattern

In [27]:
from collections import namedtuple

In [28]:
Customer = namedtuple('Customer', 'name fidelity')

In [30]:
class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

In [41]:
class Order: # the Context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion # 利用了python中函数是first-class object，赋值给变量
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

## 将策略抽象类和各个具体类 改为方法

In [42]:
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points""" 
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [43]:
def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

In [44]:
def large_order_promo(order):
    """7% discount"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07 
    return 0

In [45]:
""""""

''

# 使用函数装饰器来改进Strategy模式

In [46]:
promos = []

In [49]:
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func # 不修改被装饰函数

In [50]:
@promotion
def fidelity(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [52]:
@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

In [53]:
@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07 
    return 0

In [54]:
def best_promo(order):
    return max(promo(order) for promo in promos)